In [ ]:
# Ingest Time Series - GHG total with LULUCF, in kt CO₂ equivalent into Trino pipeline

In [ ]:
Run these in a notebook cell if you need to install onto your nb env

In [2]:
# 'capture' magic prevents long outputs from spamming your notebook
#%%capture pipoutput

# For loading predefined environment variables from files
# Typically used to load sensitive access credentials
%pip install python-dotenv

# Standard python package for interacting with S3 buckets
%pip install boto3

# Interacting with Trino and using Trino with sqlalchemy
%pip install trino sqlalchemy sqlalchemy-trino

# Pandas and parquet file i/o
%pip install pandas pyarrow fastparquet

# OS-Climate utilities to make data ingest easier
%pip install osc-ingest-tools

You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import dotenv_values, load_dotenv
import osc_ingest_trino as osc
import os
import pathlib

In [ ]:
Load Environment Variables

In [4]:
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [28]:
# use a catalog that is configured for iceberg
ingest_catalog = 'osc_datacommons_iceberg_dev'
ingest_schema = 'pcaf_sovereign_footprint'
ingest_table = 'pcaf_sovereign_unfcc'

In [29]:
import trino
from sqlalchemy.engine import create_engine

env_var_prefix = 'TRINO'

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ[f'{env_var_prefix}_USER'],
    host = os.environ[f'{env_var_prefix}_HOST'],
    port = os.environ[f'{env_var_prefix}_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ[f'{env_var_prefix}_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_iceberg_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

trino_bucket = osc.attach_s3_bucket("S3_DEV")

In [30]:
import boto3

s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
source_bucket = s3_source.Bucket(os.environ['S3_LANDING_BUCKET'])

In [8]:
Open a Trino connection using JWT for authentication

SyntaxError: invalid syntax (4253789144.py, line 1)

In [25]:
# make sure schema exists, or table creation below will fail in weird ways
sql = f"""
create schema if not exists {ingest_catalog}.{ingest_schema}
"""
qres = engine.execute(sql)
print(qres.fetchall())

[(True,)]


In [10]:
# Show available schemas to ensure trino connection is set correctly
schema_read = engine.execute(f'show schemas in {ingest_catalog}')
for row in schema_read.fetchall():
    print(row)

('aicoe_osc_demo',)
('company_data',)
('default',)
('defaultschema1',)
('demo',)
('eje_test_iceberg',)
('epa_frs',)
('epa_ghgrp',)
('epacems',)
('epacems_y95_al',)
('essd',)
('ghgrp_demo',)
('gleif',)
('gleif_mdt',)
('iceberg_demo',)
('information_schema',)
('ingest_schema',)
('iso3166',)
('itr_mdt',)
('metastore',)
('metastore_iceberg',)
('osc_corp_data',)
('pcaf_sovereign_footprint',)
('physical_risk_project',)
('pudl',)
('rmi_20210929',)
('rmi_20211120',)
('rmi_20220119',)
('rmi_utility_transition_hub',)
('sec_dera',)
('sfi_geoasset',)
('team1',)
('team2',)
('testaccessschema1',)
('testdb',)
('urgentem',)
('us_census',)
('wri',)
('wri_demo',)
('wri_dev',)
('wri_gppd',)
('wri_gppd_md',)
('wri_new',)
('wri_test',)


In [ ]:
Load GDP file (updated sporadically from https://data.worldbank.org/indicator/NY.GDP.PCAP.PP.CD)

In [35]:
import pandas as pd

ticker_file = s3_source.Object(os.environ['S3_LANDING_BUCKET'],'PCAF-sovereign-footprint/UNFCC/Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_equivalent.xlsx')
ticker_file.download_file(f'/tmp/Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_equivalent.xlsx')
%run TransposeXLS.py --config UNFCC.ini --output=UNFCC.csv 
df = pd.read_csv('UNFCC.csv')
df= df[['rec_source','data_provider','country_iso_code','country_name','validity_date','attribute','value']].dropna(subset=['value'])
df["country_iso_code"]='###'
df = df.convert_dtypes()
df


UNFCC.ini
file_list:
['/tmp/Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_equivalent.xlsx']
/tmp/Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_equivalent.xlsx
2
xls
/tmp/Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_equivalen
['0', '2']
                                                Party     Base year  \
0                                           Australia  6.154780e+05   
1                                             Austria  6.622424e+04   
2                                             Belarus  1.084728e+05   
3                                             Belgium  1.423167e+05   
4                                            Bulgaria  9.582332e+04   
5                                              Canada  5.447072e+05   
6                                             Croatia  2.493960e+04   
7                                              Cyprus  5.352055e+03   
8                                             Czechia  1.901111e+05   
9                                             Denmark  7.76

,rec_source,data_provider,country_iso_code,country_name,validity_date,attribute,value
0,Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_eq...,UNFCC,###,Australia,1990,"Time Series - GHG total with LULUCF, in kt CO₂...",615477.985815
1,Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_eq...,UNFCC,###,Austria,1990,"Time Series - GHG total with LULUCF, in kt CO₂...",66224.240765
2,Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_eq...,UNFCC,###,Belarus,1990,"Time Series - GHG total with LULUCF, in kt CO₂...",108472.815865
3,Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_eq...,UNFCC,###,Belgium,1990,"Time Series - GHG total with LULUCF, in kt CO₂...",142316.734753
4,Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_eq...,UNFCC,###,Bulgaria,1990,"Time Series - GHG total with LULUCF, in kt CO₂...",80849.65522
...,...,...,...,...,...,...,...
1496,Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_eq...,UNFCC,###,Switzerland,2018,"Time Series - GHG total with LULUCF, in kt CO₂...",45593.993643
1497,Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_eq...,UNFCC,###,Turkey,2018,"Time Series - GHG total with LULUCF, in kt CO₂...",427901.732339
1498,Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_eq...,UNFCC,###,Ukraine,2018,"Time Series - GHG total with LULUCF, in kt CO₂...",340673.901364
1499,Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_eq...,UNFCC,###,United Kingdom of Great Britain and Northern I...,2018,"Time Series - GHG total with LULUCF, in kt CO₂...",472327.984603


In [42]:
import osc_ingest_trino as osc
columnschema = osc.create_table_schema_pairs(df) 

sql = f"""
drop table if exists {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
print(sql)
qres = engine.execute(sql)
print(qres.fetchall())



drop table if exists osc_datacommons_iceberg_dev.pcaf_sovereign_footprint.pcaf_sovereign_unfcc

[(True,)]


In [43]:
tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{ingest_table}(
{columnschema}
) with (
    format = 'ORC',
    partitioning = array['country_iso_code']
)
"""
print(tabledef)
qres = engine.execute(tabledef)
print(qres.fetchall())


create table if not exists osc_datacommons_iceberg_dev.pcaf_sovereign_footprint.pcaf_sovereign_unfcc(
    rec_source varchar,
    data_provider varchar,
    country_iso_code varchar,
    country_name varchar,
    validity_date bigint,
    attribute varchar,
    value double
) with (
    format = 'ORC',
    partitioning = array['country_iso_code']
)

[(True,)]


In [44]:
# Delete all data from our db, so we start with empty table
sql=f"""
delete from {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
qres = engine.execute(sql)
print(qres.fetchall())

[(None,)]


In [45]:
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
pd.read_sql(sql, engine)


,rec_source,data_provider,country_iso_code,country_name,validity_date,attribute,value


In [46]:
print(ingest_catalog)
#df=df.drop(df[df.country_name=="cote d'ivoire"].index)
df.to_sql(ingest_table,
           con=engine,
           schema=ingest_schema,
           if_exists='append',
           index=False,
           method=osc.TrinoBatchInsert(batch_size = 100, verbose = True))

osc_datacommons_iceberg_dev
inserting 100 records
constructed fully qualified table name as: "pcaf_sovereign_footprint.pcaf_sovereign_unfcc"
batch insert result: [(100,)]
inserting 100 records
constructed fully qualified table name as: "pcaf_sovereign_footprint.pcaf_sovereign_unfcc"
batch insert result: [(100,)]
inserting 100 records
constructed fully qualified table name as: "pcaf_sovereign_footprint.pcaf_sovereign_unfcc"
batch insert result: [(100,)]
inserting 100 records
constructed fully qualified table name as: "pcaf_sovereign_footprint.pcaf_sovereign_unfcc"
batch insert result: [(100,)]
inserting 100 records
constructed fully qualified table name as: "pcaf_sovereign_footprint.pcaf_sovereign_unfcc"
batch insert result: [(100,)]
inserting 100 records
constructed fully qualified table name as: "pcaf_sovereign_footprint.pcaf_sovereign_unfcc"
batch insert result: [(100,)]
inserting 100 records
constructed fully qualified table name as: "pcaf_sovereign_footprint.pcaf_sovereign_unfcc"
b

In [47]:
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table}"""
pd.read_sql(sql, engine)


,rec_source,data_provider,country_iso_code,country_name,validity_date,attribute,value
0,Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_eq...,UNFCC,###,Switzerland,1998,"Time Series - GHG total with LULUCF, in kt CO₂...",5.088778e+04
1,Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_eq...,UNFCC,###,Turkey,1998,"Time Series - GHG total with LULUCF, in kt CO₂...",2.178076e+05
2,Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_eq...,UNFCC,###,Ukraine,1998,"Time Series - GHG total with LULUCF, in kt CO₂...",4.297564e+05
3,Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_eq...,UNFCC,###,United Kingdom of Great Britain and Northern I...,1998,"Time Series - GHG total with LULUCF, in kt CO₂...",7.549733e+05
4,Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_eq...,UNFCC,###,United States of America,1998,"Time Series - GHG total with LULUCF, in kt CO₂...",6.220470e+06
...,...,...,...,...,...,...,...
1300,Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_eq...,UNFCC,###,Germany,1994,"Time Series - GHG total with LULUCF, in kt CO₂...",1.098880e+06
1301,Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_eq...,UNFCC,###,Greece,1994,"Time Series - GHG total with LULUCF, in kt CO₂...",1.043885e+05
1302,Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_eq...,UNFCC,###,Hungary,1994,"Time Series - GHG total with LULUCF, in kt CO₂...",7.221763e+04
1303,Time_Series_GHG_total_with_LULUCF_in_kt_CO₂_eq...,UNFCC,###,Iceland,1994,"Time Series - GHG total with LULUCF, in kt CO₂...",1.256519e+04
